In [ ]:
# default_exp data.mixed

# Mixed data

> DataLoader than can take data from multiple dataloaders with different types of data

In [ ]:
#export
from tsai.imports import *

In [ ]:
#export
# This implementation of a mixed dataloader is based on a great implementation created by Zach Mueller in this fastai thread:
# https://forums.fast.ai/t/combining-tabular-images-in-fastai2-and-should-work-with-almost-any-other-type/73197

from packaging import version
from fastai.data.load import _FakeLoader
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)

class MixedDataLoaders():
    def __init__(self, *dls, device=None, bs=None):
        "Accepts any number of `DataLoaders` and a device"
        device = ifnone(device, default_device())
        self.device = device
        self.c = None
        bs = ifnone(bs, min([dl.bs for dl in dls]))
        for i, dl in enumerate(dls): # ensure all dls have the same bs
            dl.bs = bs
            dl.shuffle_fn = self.shuffle_fn
            if self.c is None and hasattr(dl, "c"): self.c = dl.c
            if i == 0: self.dataset = dl.dataset
            dl.to(device=device)
        self.dls = dls
        self.count = 0
        self.fake_l = _FakeLoader(self, False, 0, 0, 0) if version.parse(fastai.__version__) >= version.parse("2.1") else _FakeLoader(self, False, 0, 0)
        self._get_idxs()
        
    def __len__(self): return len(self.dls[0])
    
    def _get_vals(self, x):
        "Checks for duplicates in batches"
        idxs, new_x = [], []
        for i, o in enumerate(x): x[i] = o.cpu().numpy().flatten()
        for idx, o in enumerate(x):
            if not self._arrayisin(o, new_x):
                idxs.append(idx)
                new_x.append(o)
        return idxs
    
    def _get_idxs(self):
        "Get `x` and `y` indices for batches of data"
        self.n_inps = [dl.n_inp for dl in self.dls]
        self.x_idxs = self._split_idxs(self.n_inps)
        
        # Identify duplicate targets
        dl_dict = dict(zip(range(0,len(self.dls)), self.n_inps))
        outs = L([])
        for key, n_inp in dl_dict.items():
            b = next(iter(self.dls[key]))
            outs += L(b[n_inp:])
        self.y_idxs = self._get_vals(outs)    
    
    def __iter__(self):
        z = zip(*[_loaders[i.fake_l.num_workers==0](i.fake_l) for i in self.dls])
        for b in z:  
            inps = []
            outs = []
            if self.device is not None: b = to_device(b, self.device)
            for batch, dl in zip(b, self.dls):
                batch = dl.after_batch(batch)
                inps += batch[:dl.n_inp]
                outs += batch[dl.n_inp:]
            inps = tuple([tuple(L(inps)[idx]) if isinstance(idx, list) else inps[idx] for idx in self.x_idxs]) if len(self.x_idxs) > 1 else tuple(L(outs)[self.x_idxs][0])
            outs = tuple(L(outs)[self.y_idxs]) if len(self.y_idxs) > 1 else L(outs)[self.y_idxs][0]
            yield inps, outs
            
    
    def one_batch(self):
        "Grab one batch of data"
        with self.fake_l.no_multiproc(): res = first(self)
        if hasattr(self, 'it'): delattr(self, 'it')
        return res

    def shuffle_fn(self, idxs):
        "Generate the same idxs for all dls in each batch"
        if self.count == 0: self.rng = self.dls[0].rng.sample(idxs, len(idxs))
        self.count += 1
        if self.count == len(self.dls): self.count = 0
        return self.rng

    def show_batch(self):
        "Show a batch of data"
        for dl in self.dls: dl.show_batch()
            
    def to(self, device): self.device = device

    def _arrayisin(self, arr, arr_list):
        "Checks if `arr` is in `arr_list`"
        for a in arr_list:
            if np.array_equal(arr, a): return True
        return False
    
    def _split_idxs(self, a):
        a_cum = np.array(a).cumsum().tolist()
        b = np.arange(sum(a)).tolist()
        start = 0
        b_ = []
        for i, idx in enumerate(range(len(a))):
            end = a_cum[i]
            b_.append(b[start:end] if end - start > 1 else b[start])
            start = end
        return b_

In [ ]:
#export
def get_mixed_dls(*dls, device=None, **kwargs):
    device = ifnone(device, default_device())
    _mixed_train_dls = []
    _mixed_valid_dls = []
    for dl in dls:
        _mixed_train_dls.append(dl.train)
        _mixed_valid_dls.append(dl.valid)
    mixed_train_dl = MixedDataLoaders(*_mixed_train_dls, **kwargs)
    mixed_valid_dl = MixedDataLoaders(*_mixed_valid_dls, **kwargs)
    mixed_dls = DataLoaders(mixed_train_dl, mixed_valid_dl, device=device)
    return mixed_dls

In [ ]:
#hide
#original
# class MixedDataLoaders():
#     def __init__(self, *dls, device='cuda:0'):
#         "Accepts any number of `DataLoaders` and a device"
#         self.device = device
#         for dl in dls: dl.shuffle_fn = self.shuffle_fn
#         self.dls = dls
#         self.count = 0
#         self.fake_l = _FakeLoader(self, False, 0, 0, 0) if version.parse(fastai.__version__) >= version.parse("2.1") else _FakeLoader(self, False, 0, 0)
#         self._get_idxs()
        
#     def __len__(self): return len(self.dls[0])
    
#     def _get_vals(self, x):
#         "Checks for duplicates in batches"
#         idxs, new_x = [], []
#         for i, o in enumerate(x): x[i] = o.cpu().numpy().flatten()
#         for idx, o in enumerate(x):
#             if not _arrayisin(o, new_x):
#                 idxs.append(idx)
#                 new_x.append(o)
#         return idxs
    
    
#     def _get_idxs(self):
#         "Get `x` and `y` indicies for batches of data"
#         dl_dict = dict(zip(range(0,len(self.dls)), [dl.n_inp for dl in self.dls]))
#         inps = L([])
#         outs = L([])
#         for key, n_inp in dl_dict.items():
#             b = next(iter(self.dls[key]))
#             inps += L(b[:n_inp])
#             outs += L(b[n_inp:])
#         self.x_idxs = self._get_vals(inps)
#         self.y_idxs = self._get_vals(outs)
        
    
#     def __iter__(self):
#         z = zip(*[_loaders[i.fake_l.num_workers==0](i.fake_l) for i in self.dls])
#         for b in z:   
#             inps = []
#             outs = []
#             if self.device is not None: 
#                 b = to_device(b, self.device)
#             for batch, dl in zip(b, self.dls):
#                 batch = dl.after_batch(batch)
#                 inps += batch[:dl.n_inp]
#                 outs += batch[dl.n_inp:]
#             inps = tuple(L(inps)[self.x_idxs])
#             outs = tuple(L(outs)[self.y_idxs]
#             yield (inps, outs)
                
#     def one_batch(self):
#         "Grab one batch of data"
#         with self.fake_l.no_multiproc(): res = first(self)
#         if hasattr(self, 'it'): delattr(self, 'it')
#         return res
    
#     def shuffle_fn(self, idxs):
#         "Shuffle the internal `DataLoaders`"
#         if self.count == 0:
#             self.rng = self.dls[0].rng.sample(idxs, len(idxs))
#             self.count += 1
#             return self.rng
#         if self.count == 1:
#             self.count = 0
#             return self.rng

        
#     def show_batch(self):
#         "Show a batch of data"
#         for dl in self.dls:
#             dl.show_batch()
            
#     def to(self, device): self.device = device
        
        
# def _arrayisin(arr, arr_list):
#     "Checks if `arr` is in `arr_list`"
#     for a in arr_list:
#         if np.array_equal(arr, a):
#             return True
#     return False

In [ ]:
from tsai.data.tabular import *

path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
# df['salary'] = np.random.rand(len(df)) # uncomment to simulate a cont dependent variable
target = 'salary'
splits = RandomSplitter()(range_of(df))

cat_names = ['workclass', 'education', 'marital-status']
cont_names = ['age', 'fnlwgt']
dls1 = get_tabular_dls(df, cat_names=cat_names, cont_names=cont_names, y_names=target, splits=splits, bs=512)
dls1.show_batch()

cat_names = None #['occupation', 'relationship', 'race']
cont_names = ['education-num']
dls2 = get_tabular_dls(df, cat_names=cat_names, cont_names=cont_names, y_names=target, splits=splits, bs=128)
dls2.show_batch()

,workclass,education,marital-status,age,fnlwgt,salary
0,Private,10th,Divorced,32.000000,177530.999753,<50k
1,Private,Bachelors,Never-married,22.000000,117779.001813,<50k
2,Private,HS-grad,Married-civ-spouse,33.000000,222220.999014,>=50k
3,Private,HS-grad,Divorced,26.000000,365288.996370,<50k
4,?,10th,Separated,29.000000,78528.996027,<50k
5,Private,Some-college,Never-married,21.000001,275189.999687,<50k
6,Local-gov,Masters,Never-married,33.000000,162623.000206,<50k
7,Self-emp-not-inc,HS-grad,Married-civ-spouse,52.000000,89028.003074,<50k
8,Private,HS-grad,Never-married,47.000000,160647.000200,<50k
9,?,Some-college,Never-married,22.000000,219940.999007,<50k


,education-num_na,education-num,salary
0,False,9.0,<50k
1,False,11.0,>=50k
2,False,13.0,<50k
3,False,13.0,<50k
4,False,10.0,<50k
5,False,9.0,<50k
6,False,12.0,<50k
7,False,9.0,<50k
8,False,10.0,<50k
9,False,10.0,>=50k


In [ ]:
dls = get_mixed_dls(dls1, dls2, bs=8)
dls.train.show_batch()

,workclass,education,marital-status,age,fnlwgt,salary
0,Private,Prof-school,Married-civ-spouse,51.000000,310773.999128,>=50k
1,Private,Some-college,Never-married,23.000000,238831.001257,<50k
2,Private,Some-college,Divorced,40.000000,34722.003867,>=50k
3,Private,Bachelors,Never-married,35.000000,226311.000392,<50k
4,Private,Some-college,Never-married,27.000000,347152.995158,<50k
5,Private,11th,Never-married,17.999999,242892.999075,<50k
6,Private,Some-college,Married-civ-spouse,30.000000,210851.000098,<50k
7,Private,Some-college,Married-civ-spouse,61.000000,85194.000538,<50k


,education-num_na,education-num,salary
0,False,15.0,>=50k
1,False,10.0,<50k
2,False,10.0,>=50k
3,False,13.0,<50k
4,False,10.0,<50k
5,False,7.0,<50k
6,False,10.0,<50k
7,False,10.0,<50k


In [ ]:
xb, yb = first(dls.train)
xb

((tensor([[ 5, 15,  3],
          [ 5, 16,  5],
          [ 5, 16,  1],
          [ 5, 10,  5],
          [ 5, 16,  5],
          [ 5,  2,  5],
          [ 5, 16,  3],
          [ 5, 16,  3]]),
  tensor([[ 0.9115,  1.1440],
          [-1.1420,  0.4624],
          [ 0.1048, -1.4713],
          [-0.2619,  0.3438],
          [-0.8486,  1.4886],
          [-1.5087,  0.5009],
          [-0.6286,  0.1973],
          [ 1.6449, -0.9932]])),
 (tensor([[1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1],
          [1]]),
  tensor([[ 1.9291],
          [-0.0350],
          [-0.0350],
          [ 1.1435],
          [-0.0350],
          [-1.2134],
          [-0.0350],
          [-0.0350]])))

In [ ]:
xs, ys = first(dls.train)
xs[0][0].shape, xs[0][1].shape, xs[1][0].shape, xs[1][1].shape

(torch.Size([8, 3]),
 torch.Size([8, 2]),
 torch.Size([8, 1]),
 torch.Size([8, 1]))

In [ ]:
#hide
out = create_scripts(); beep(out)